In [75]:
import requests
import json
import pandas as pd
from datetime import datetime

# Set query (which uses text input to specify ETH wallet addresses)
query = '''query {
  swaps(first:500 orderBy: timestamp orderDirection: desc){
    id
    timestamp
    pool{
      token0{
        symbol
      }
      token1{
        symbol
      }
      txCount
    }
    amount0
    amount1
    amountUSD
    token0{
        id
        totalSupply
        volume
        volumeUSD
        totalValueLocked
        totalValueLockedUSD
    }
    token1{
        id
        totalSupply
        volume
        volumeUSD
        totalValueLocked
        totalValueLockedUSD
    }
  }
}
''' 
#% address

# Point to correct subgraph URL
url = 'https://api.thegraph.com/subgraphs/name/benesjan/uniswap-v3-subgraph'
# Make the request
r = requests.post(url, json={'query': query})
#print(r.status_code)
#print(r.text)

# JSON adjustment
json_data = json.loads(r.text)

# extract JSON to convert to a dataframe
df_data = json_data['data']['swaps']
# convert to dataframe
df1 = pd.DataFrame(df_data)  

#print(df1)
#EDA
#df1.to_excel(r'C:\Users\Nehal.Jain\Documents\Nehal_Personal\Project\Blockchain\Dataset.xlsx', sheet_name='Data', index = False)

#Removing NULLS - Data Cleaning
df = df1.dropna()
#print(df)

#print(df.isnull().values.any())
#print(df.isnull().sum().sum())

#Converting from Unix timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')

#Splitting 'pool' column into 'From_Currency' and 'To_Currency'

df[['From_Currency','To_Currency','txCount']] = df.pool.apply(
   lambda x: pd.Series(str(x).split(",")))

#Splitting 'Token' columns
df[['From_Token_id','From_Token_totalSupply','From_Token_totalValueLocked','From_Token_totalValueLockedUSD','From_Token_volume','From_Token_volumeUSD']] = df.token0.apply(
   lambda z: pd.Series(str(z).split(",")))
df[['To_Token_id','To_Token_totalSupply','To_Token_totalValueLocked','To_Token_totalValueLockedUSD','To_Token_volume','To_Token_volumeUSD']] = df.token1.apply(
   lambda n: pd.Series(str(n).split(",")))

#Removing extra data from columns From_Currency and To_Currency : Cleaning
df['From_Currency'] = df['From_Currency'].map(lambda x: str(x)[23:-2])
df['To_Currency'] = df['To_Currency'].map(lambda y: str(y)[23:-2])
df['txCount'] = df['txCount'].map(lambda y: str(y)[13:-2])
df['From_Token_id'] = df['From_Token_id'].map(lambda y: str(y)[8:-1])
df['To_Token_id'] = df['To_Token_id'].map(lambda y: str(y)[8:-1])
df['From_Token_totalSupply'] = df['From_Token_totalSupply'].map(lambda y: str(y)[17:-1])
df['To_Token_totalSupply'] = df['To_Token_totalSupply'].map(lambda y: str(y)[17:-1])
df['From_Token_totalValueLocked'] = df['From_Token_totalValueLocked'].map(lambda y: str(y)[22:-1])
df['From_Token_totalValueLockedUSD'] = df['From_Token_totalValueLockedUSD'].map(lambda y: str(y)[25:-1])
df['To_Token_totalValueLocked'] = df['To_Token_totalValueLocked'].map(lambda y: str(y)[22:-1])
df['To_Token_totalValueLockedUSD'] = df['To_Token_totalValueLockedUSD'].map(lambda y: str(y)[25:-1])
df['From_Token_volume'] = df['From_Token_volume'].map(lambda y: str(y)[12:-1])
df['From_Token_volumeUSD'] = df['From_Token_volumeUSD'].map(lambda y: str(y)[15:-2])
df['To_Token_volume'] = df['To_Token_volume'].map(lambda y: str(y)[12:-1])
df['To_Token_volumeUSD'] = df['To_Token_volumeUSD'].map(lambda y: str(y)[15:-2])

#print(df)

#Dropping the pool, token columns
df = df.drop(['pool'], axis=1)
df = df.drop(['token0'], axis=1)
df = df.drop(['token1'], axis=1)

#Cleaning and re-ordering of data
df = df.rename(columns={'amount0': 'From_Currency_Amount', 'amount1': 'To_Currency_Amount', 'amountUSD': 'Amount_USD', 'id': 'TransactionID','timestamp': 'Date_Time'})
df = df [['TransactionID','From_Token_id','From_Currency_Amount','From_Currency','From_Token_totalSupply','From_Token_totalValueLocked','From_Token_totalValueLockedUSD','From_Token_volume','From_Token_volumeUSD','To_Token_id','To_Currency_Amount','To_Currency','To_Token_totalSupply','To_Token_totalValueLocked','To_Token_totalValueLockedUSD','To_Token_volume','To_Token_volumeUSD','Amount_USD','txCount','Date_Time']]
df.head()
#print(df)


,TransactionID,From_Token_id,From_Currency_Amount,From_Currency,From_Token_totalSupply,From_Token_totalValueLocked,From_Token_totalValueLockedUSD,From_Token_volume,From_Token_volumeUSD,To_Token_id,To_Currency_Amount,To_Currency,To_Token_totalSupply,To_Token_totalValueLocked,To_Token_totalValueLockedUSD,To_Token_volume,To_Token_volumeUSD,Amount_USD,txCount,Date_Time
0,0xdfc10463ed07358905e115b0c664087a493310968ede...,0x90de74265a416e1393a450752175aed98fe11517,14.660925869687611572,UDT,28240,1184.62471553910271931,0,43172.760948599151524103,15834609.66543299940731477701356362,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,-1.286376713345159283,WETH,19848,249785.019858581600064453,589573764.7031978988224949163055534,27044455.025337035678342349,64692801149.03132087315877621037512,3036.266795113718122563024022254934,4378,2021-07-26 17:15:58
1,0xc049018afd69a8440f775ae1cfd380700c68528cbac6...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,1.450000000000000512,WETH,19848,249785.019858581600064453,589573764.7031978988224949163055534,27044455.025337035678342349,64692801149.03132087315877621037512,0xdac17f958d2ee523a2206206994597c13d831ec7,-3416.316539,USDT,18240,191396323.169713,191363082.0678834557017059285480571,17565547787.300604,17579434702.45684206818665854104018,3419.09704622036318370377433151338,231915,2021-07-26 17:15:58
2,0xc049018afd69a8440f775ae1cfd380700c68528cbac6...,0x07bac35846e5ed502aa91adf6a9e7aa210f2dcbe,-20560.933499271904146928,erowan,18240,486219.724630969799069137,0,17261256.343735316132294337,3546819.412652176338067118279593667,0xdac17f958d2ee523a2206206994597c13d831ec7,3416.316539,USDT,18240,191396323.169713,191363082.0678834557017059285480571,17565547787.300604,17579434702.45684206818665854104018,3415.723204059839427048994486327271,2142,2021-07-26 17:15:58
3,0x53afb5341d99fb376f14b8b90bd806b9260ad7dda808...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,97197.78488,USDC,19312,470025748.715368,470025748.7153680000000000000000001,44832439441.278256,44833497658.69235132401420594728058,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,-41.218173527585695362,WETH,19848,249785.019858581600064453,589573764.7031978988224949163055534,27044455.025337035678342349,64692801149.03132087315877621037512,97243.0300159978349973016596560902,374716,2021-07-26 17:15:58
4,0xc10100c6827e965004f9ad33159eb7454558bf5ddbc1...,0x33f391f4c4fe802b70b77ae37670037a92114a7c,-2000,BURP,28240,955045.624592500641772063,0,6927858.854043058790626265,2264780.864098544109644973621162435,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0.235133411313485368,WETH,19848,249785.019858581600064453,589573764.7031978988224949163055534,27044455.025337035678342349,64692801149.03132087315877621037512,554.9912104179948105170551305151725,640,2021-07-26 17:15:56
